In [36]:
"""
计算基础包导入
"""
import numpy as np
import pandas as pd

In [37]:
"""
该cell加载非计算内容, 仅用于展示结果
"""
from rich.table import Table
from rich.console import Console

console = Console()

def print_tidy(info_list: list, title: str = ''):
    table = Table(show_header=False, pad_edge=False, box=None, title=title)

    # 添加列（justify参数控制对齐方式）
    table.add_column(justify="left")
    table.add_column(justify="left")

    # 添加数据
    for name, price in info_list:
        table.add_row(name, str(price))

    console.print(table)

读取数据源  
目前通过SCV获取股票数据

In [ ]:
# 数据源每一列的含义
# 股票编号
TS_CODE = 0
TRADE_DATE = 1
OPEN = 2
HIGH = 3
LOW = 4
CLOSE = 5
PRE_CLOSE = 6
CHANGE = 7
PCT_CHG = 8
VOL = 9
AMOUNT = 10

In [29]:
# 时间跨度为一年的数据
file_name = r'D:\workplace\datasources\300059.SZ.year.csv'
high_price, low_price, vol, close_price = np.loadtxt(
    fname = file_name,
    delimiter=',',
    usecols = (HIGH, LOW, VOL, CLOSE),
    unpack = True
)

# 时间跨度是一个月的数据
file_name = r'D:\workplace\datasources\300059.SZ.month.csv'
high_price_m, low_price_m, vol_m, close_price_m = np.loadtxt(
    fname = file_name,
    delimiter=',',
    usecols = (HIGH, LOW, VOL, CLOSE),
    unpack = True
)

# 极差
股价近期最高价的最大值与最小值的差值
差值越大说明波动越明显

In [ ]:
# ptp(Peak-to-Peak) 最大峰值 - 最小峰值 = 极差
hight_ptp = np.ptp(high_price)
low_ptp = np.ptp(low_price)

print(np.round(hight_ptp, 2), np.round(low_ptp, 2))

30.38 28.93


# 成交量加权平均价格
Volume-Weighted Average Price(VWAP)  
一个非常重要的经济学指标，代表着金融资产的“平均”价格

In [16]:
info_list = [
    ('中位价格', np.round(np.median(close_price), 2)),
    ('平均价格', np.round(np.average(close_price), 2)),
    ('加权平均价格', np.round(np.average(close_price, weights=vol), 2))
]
print_tidy(info_list)

中位价格      22.52
平均价格      22.4 
加权平均价格  23.04

# 收益率
简单收益率： 相邻两个价格之间的变化率  
对数收益率： 指对所有价格取对数后两两之间的差值

In [ ]:
def volatility_caculation(_close_price, title):
    # 计算 对数收益率
    # 数学意义 log(a) - log(b) = log(a/b)
    # 即相邻收盘价对数的差值
    # log_return = np.diff(np.log(_close_price))

    # 通过pandas的Series实现对数收益率的计算的好处是可以保留索引，假如数据的key是日期，
    # 那么所计算的收益率所属日期也会被保留下来，这样后续与其他数据对齐时将非常方便
    close_price_series = pd.Series(_close_price)
    log_return = np.log(close_price_series / close_price_series.shift(1)).dropna()

    # 日收益率
    daily_volatility = log_return.std()

    # 月收益率
    monthly_volatility = daily_volatility * np.sqrt(21)

    # 年收益率
    annual_volatility = daily_volatility * np.sqrt(250)


    info_list = [
        ('日收益率', np.round(daily_volatility, 4)),
        ('月收益率', np.round(monthly_volatility, 4)),
        ('年收益率', np.round(annual_volatility, 4))
    ]
    print_tidy(info_list, title)

volatility_caculation(close_price, '基于年数据')
volatility_caculation(close_price_m, '基于月数据')


   基于年数据   
日收益率  0.0383
月收益率  0.1754
年收益率  0.6052

   基于月数据   
日收益率  0.0193
月收益率  0.0885
年收益率  0.3054

# 计算过程分析(年收益率)
## 第一步： 收盘价对数化
对应代码块 `np.log(close_price)`  
本质上就是对收盘价列表中每一个价格取对数 a -> $log(a)$

## 第二步： 计算对数收益率
对应代码块`log_return = np.diff(np.log(close_price))`  
本质上就是取相邻对数的差值，获取一个长度减少一位的新列表，即`len(close_price) - len(log_return) == 1`。  
其中`log_return`列表中每个参数都是`close_price`中$log(b) - log(a)$计算得来。

## 第三步： 计算对数收益率的标准差(日波动率)
对应代码块`log_return.std()`  
计算的是对数收益率序列的标准差。标准差主要用于衡量数据的离散程度，在金融领域，它反映了资产收益率的波动程度。  
对应数学公式：  
$σ = \sqrt{\frac{1}{N}\sum_{i=1}^{N}(x_i-\overline{x})^2}$  
$x_i$ - 每一个对数收益率  
$\overline{x}$ - 对数收益率的平均值  
$N$ - 样本数量  
标准差越大，表明收益率的波动越大，资产的风险也越大

## 第四步： 计算年收益率
对应代码块`annual_volatility = log_return.std() * np.sqrt(250)`  
本质上是将日波动率转换为年波动率。之所以乘以$\sqrt{250}$是因为：  
- 通常认为一年的交易日大约为250天。  
- 当收益率相互独立时，波动率会随着时间的平方根增长，这是因为**方差（标准差的平方）具有可加性**。如果日收益率的方差是$σ_{day}^2$，那么T天的方差就是$T\timesσ_{day}^2$，相应方差就是$σ_{day}\times\sqrt{T}$

> - 上述计算过程假设了对数收益率序列满足独立同分布(IID)以及正太分布。但实际金融市场中，收益率可能存在自相关性、异方差性等情况。
> - 一年的交易数量（250天）只是一个常用的近似值，在不同的市场中可能会有所不同。
> - 该计算得到的是历史波动率，它反应的是过去的风险情况，未来的实际波动率可能会有所不同。
